<center>
<img src="../../img/ods_stickers.jpg" />
    
## [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 
Author: [Yury Kashnitskiy](https://yorko.github.io) (@yorko). Edited by Sergey Kolchenko (@KolchenkoSergey). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose.

## <center>Assignment #6
### <center> Beating baselines in "How good is your Medium article?"
    
<img src='../../img/medium_claps.jpg' width=40% />


[Competition](https://www.kaggle.com/c/how-good-is-your-medium-article). The task is to beat "A6 baseline" (~1.45 Public LB score). Do not forget about our shared ["primitive" baseline](https://www.kaggle.com/kashnitsky/ridge-countvectorizer-baseline) - you'll find something valuable there.

**Your task:**
 1. "Freeride". Come up with good features to beat the baseline "A6 baseline" (for now, public LB is only considered)
 2. You need to name your [team](https://www.kaggle.com/c/how-good-is-your-medium-article/team) (out of 1 person) in full accordance with the [course rating](https://drive.google.com/open?id=19AGEhUQUol6_kNLKSzBsjcGUU3qWy3BNUg8x8IFkO3Q). You can think of it as a part of the assignment. 16 credits for beating the mentioned baseline and correct team naming.

In [1]:
import os
import json
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import Ridge

The following code will help to throw away all HTML tags from an article content.

In [2]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

Supplementary function to read a JSON line without crashing on escape characters.

In [3]:
def read_json_line(line=None):
    result = None
    try:        
        result = json.loads(line)
    except Exception as e:      
        # Find the offending character index:
        idx_to_replace = int(str(e).split(' ')[-1].replace(')',''))      
        # Remove the offending character:
        new_line = list(line)
        new_line[idx_to_replace] = ' '
        new_line = ''.join(new_line)     
        return read_json_line(line=new_line)
    return result

Extract features `content`, `published`, `title` and `author`, write them to separate files for train and test sets.

In [77]:
def extract_features_and_write(path_to_data,
                               inp_filename, is_train=True):
    
    features = ['content', 'published', 'title', 'author']
    prefix = 'train' if is_train else 'test'
    feature_files = [open(os.path.join(path_to_data,
                                       '{}_{}.txt'.format(prefix, feat)),
                          'w', encoding='utf-8')
                     for feat in features]
    
    with open(os.path.join(path_to_data, inp_filename), 
              encoding='utf-8') as inp_json_file:

        for line in tqdm_notebook(inp_json_file):
            json_data = read_json_line(line)
            
            # You code here
            for i in range(len(features)):
                content = str(json_data[features[i]]).replace('\n', ' ').replace('\r', ' ')
                content_no_html_tags = strip_tags(content)
                feature_files[i].write(str(content_no_html_tags) + '\n')
                #feature_files[i].write(str(json_data[features[i]]) + '\n')

In [78]:
PATH_TO_DATA = '../../data/kaggle_medium' # modify this if you need to

In [79]:
#os.listdir()

In [80]:
extract_features_and_write(PATH_TO_DATA, 'train.json', is_train=True)

In [81]:
extract_features_and_write(PATH_TO_DATA, 'test.json', is_train=False)

**Add the following groups of features:**
    - Tf-Idf with article content (ngram_range=(1, 2), max_features=100000 but you can try adding more)
    - Tf-Idf with article titles (ngram_range=(1, 2), max_features=100000 but you can try adding more)
    - Time features: publication hour, whether it's morning, day, night, whether it's a weekend
    - Bag of authors (i.e. One-Hot-Encoded author names)

In [82]:
# You code here
def TfIdf(path_to_data, inp_filename):
    with open(os.path.join(path_to_data, inp_filename), 
              encoding='utf-8') as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    vectorizer = TfidfVectorizer(max_features=100000, ngram_range=(1,2))
    X = vectorizer.fit_transform(content)
    print(vectorizer.get_feature_names()[:10])
#     print(X.shape)
    return X, vectorizer

In [84]:
X_train_content_sparse, vect_content = TfIdf(PATH_TO_DATA, 'train_content.txt')
X_train_title_sparse, vect_title = TfIdf(PATH_TO_DATA, 'train_title.txt')
#print(X_train_title_sparse.shape)

['00', '00 00', '00 am', '00 pm', '000', '000 00', '000 000', '000 and', '000 euros', '000 feet']
['00', '000', '000 000', '000 at', '000 blog', '000 day', '000 for', '000 google', '000 in', '000 miles']


In [85]:
print(vect_content.get_feature_names()[:10])
print(vect_title.get_feature_names()[:10])
#     print(X.shape)

['00', '00 00', '00 am', '00 pm', '000', '000 00', '000 000', '000 and', '000 euros', '000 feet']
['00', '000', '000 000', '000 at', '000 blog', '000 day', '000 for', '000 google', '000 in', '000 miles']


In [86]:
import scipy.sparse
scipy.sparse.save_npz('../../data/kaggle_medium/train_title_matrix.npz',X_train_title_sparse) 

In [87]:
scipy.sparse.save_npz('../../data/kaggle_medium/train_content_matrix.npz',X_train_content_sparse) 

In [88]:
X_train_content_sparse.shape

(62313, 100000)

In [89]:
X_train_title_sparse.shape

(62313, 100000)

In [90]:
def TfIdf_test(path_to_data, inp_filename, vectorizer):
    with open(os.path.join(path_to_data, inp_filename), 
              encoding='utf-8') as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    
    X = vectorizer.transform(content)
    #print(vectorizer.get_feature_names()[:10])
#     print(X.shape)
    return X

In [91]:
X_test_content_sparse = TfIdf_test(PATH_TO_DATA, 'test_content.txt', vect_content)
X_test_title_sparse = TfIdf_test(PATH_TO_DATA, 'test_title.txt', vect_title)
scipy.sparse.save_npz('../../data/kaggle_medium/test_title_matrix.npz', X_test_title_sparse)
scipy.sparse.save_npz('../../data/kaggle_medium/test_content_matrix.npz', X_test_content_sparse)

In [92]:
print(X_test_content_sparse.shape)

(34645, 100000)


In [93]:
print(X_test_title_sparse.shape)

(34645, 100000)


In [94]:
def timeFeature(path_to_data, inp_filename):
    df_published = pd.read_csv(os.path.join(path_to_data, inp_filename),
                             sep = " ", header = None)
    df_published.columns = ['tag', 'date']
    df_published.date = df_published.date.str.replace(r'}$', '')
    df_published['date'] = pd.to_datetime(df_published['date'])
    df_published['hour'] = df_published.date.dt.hour
    df_published['morning'] = (df_published['hour'] <= 9) & (df_published['hour'] >= 6)
    df_published['day'] = (df_published['hour'] <= 18) & (df_published['hour'] > 9)
    df_published['night'] = (df_published['hour'] > 18) | (df_published['hour'] < 6)
    df_published['isWeekend'] = df_published.date.dt.dayofweek >= 5
    df_published = df_published.drop(columns=['tag','date'])
    return df_published.values

In [95]:
X_train_time_features_sparse = timeFeature(PATH_TO_DATA, 'train_published.txt')
X_train_time_features_sparse

array([[22, False, False, True, False],
       [7, True, False, False, False],
       [13, False, True, False, True],
       ...,
       [6, True, False, False, True],
       [17, False, True, False, True],
       [3, False, False, True, False]], dtype=object)

In [96]:
print(X_train_time_features_sparse.shape)

(62313, 5)


In [97]:
X_test_time_features_sparse = timeFeature(PATH_TO_DATA, 'test_published.txt')
print(X_test_time_features_sparse.shape)

(34645, 5)


In [45]:
# def authorFeature(path_to_data, inp_filename):
#     df_author = pd.read_csv(os.path.join(PATH_TO_DATA, inp_filename), sep=' ', header = None)
#     df_author.columns = ['tag0', 'tag1', 'tag2', 'tag3', 'tag4', 'twitter']
#     df_author.twitter = df_author.twitter.str.replace(r'}$', '')
#     one_hot = pd.get_dummies(df_author['twitter'])
#     df_author = df_author.join(one_hot)
#     df_author = df_author.drop(columns=['tag0', 'tag1','tag2','tag3','tag4','twitter'])
#     return df_author.values

In [98]:
df_author = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_author.txt'), sep=' ', header = None)
df_author.columns = ['tag0', 'tag1', 'tag2', 'tag3', 'tag4', 'twitter']
df_author.twitter = df_author.twitter.str.replace(r'}$', '')
cat_columns = ['twitter']
df_author_processed = pd.get_dummies(df_author, prefix_sep="__", columns=cat_columns)
df_author_processed.head()

,tag0,tag1,tag2,tag3,tag4,twitter__'@0723Sam',twitter__'@0canom',twitter__'@0guzKilic',twitter__'@0ptionworkshop',twitter__'@0rf',...,twitter__'@zumpang',twitter__'@zvellas',twitter__'@zverk0',twitter__'@zysman',twitter__'@zzam_',twitter__'@zzbennett',twitter__'@zzdoublezz',twitter__'@zzilch',twitter__'@zzste',twitter__None
0,{'name':,"None,",'url':,"'https://medium.com/@Medium',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,{'name':,"None,",'url':,"'https://medium.com/@Medium',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,{'name':,"None,",'url':,"'https://medium.com/@aelcenganda',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,{'name':,"None,",'url':,"'https://medium.com/@vaibhavkhulbe',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,{'name':,"None,",'url':,"'https://medium.com/@vaibhavkhulbe',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
cat_dummies = [col for col in df_author_processed 
               if "__" in col 
               and col.split("__")[0] in cat_columns]
# cat_dummies

In [100]:
processed_columns = list(df_author_processed.columns[:])

In [101]:
df_author_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_author.txt'), sep=' ', header = None)
df_author_test.columns = ['tag0', 'tag1', 'tag2', 'tag3', 'tag4', 'twitter']
df_author_test.twitter = df_author_test.twitter.str.replace(r'}$', '')
df_author_test.head()

,tag0,tag1,tag2,tag3,tag4,twitter
0,{'name':,"None,",'url':,"'https://medium.com/@HITRECORD.org',",'twitter':,'@hitRECord'
1,{'name':,"None,",'url':,"'https://medium.com/@mariabustillos',",'twitter':,'@mariabustillos'
2,{'name':,"None,",'url':,"'https://medium.com/@HITRECORD.org',",'twitter':,'@hitRECord'
3,{'name':,"None,",'url':,"'https://medium.com/@LanceUlanoff',",'twitter':,'@LanceUlanoff'
4,{'name':,"None,",'url':,"'https://blog.medium.com/@Medium',",'twitter':,'@Medium'


In [102]:
df_test_processed = pd.get_dummies(df_author_test, prefix_sep="__", 
                                   columns=cat_columns)
df_test_processed.head()

,tag0,tag1,tag2,tag3,tag4,twitter__'@007_drive',twitter__'@02215507',twitter__'@08181',twitter__'@0guzKilic',twitter__'@0x0ece',...,twitter__'@zntfdr',twitter__'@zoeschlag',twitter__'@zopsesen',twitter__'@zoyander',twitter__'@zs',twitter__'@zslayback',twitter__'@ztsamudzi',twitter__'@zuosisi44',twitter__'@zvona',twitter__None
0,{'name':,"None,",'url':,"'https://medium.com/@HITRECORD.org',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,{'name':,"None,",'url':,"'https://medium.com/@mariabustillos',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,{'name':,"None,",'url':,"'https://medium.com/@HITRECORD.org',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,{'name':,"None,",'url':,"'https://medium.com/@LanceUlanoff',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,{'name':,"None,",'url':,"'https://blog.medium.com/@Medium',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
# Remove additional columns
for col in df_test_processed.columns:
    if ("__" in col) and (col.split("__")[0] in cat_columns) and col not in cat_dummies:
        print("Removing additional feature {}".format(col))
        df_test_processed.drop(col, axis=1, inplace=True)
        
df_test_processed.head()

Removing additional feature twitter__'@007_drive'
Removing additional feature twitter__'@02215507'
Removing additional feature twitter__'@08181'
Removing additional feature twitter__'@0x0ece'
Removing additional feature twitter__'@0x1ad2'
Removing additional feature twitter__'@0x49D1'
Removing additional feature twitter__'@0x736A'
Removing additional feature twitter__'@0xdeadbabe'
Removing additional feature twitter__'@0zne'
Removing additional feature twitter__'@100ResCities'
Removing additional feature twitter__'@14prakash'
Removing additional feature twitter__'@196Nong'
Removing additional feature twitter__'@1eaguilera'
Removing additional feature twitter__'@1k_experiment'
Removing additional feature twitter__'@215Blog'
Removing additional feature twitter__'@215Sport'
Removing additional feature twitter__'@24thST'
Removing additional feature twitter__'@27Ajain'
Removing additional feature twitter__'@27_gerry'
Removing additional feature twitter__'@2ndlifetravel1'
Removing additional

Removing additional feature twitter__'@AmaechiPerform'
Removing additional feature twitter__'@AmaniOurs'
Removing additional feature twitter__'@Amartweetsimmor'
Removing additional feature twitter__'@AmberCSnider'
Removing additional feature twitter__'@AmberHorsburgh'
Removing additional feature twitter__'@AmeyPandit1'
Removing additional feature twitter__'@Amitch5903'
Removing additional feature twitter__'@AmnistiaOnline'
Removing additional feature twitter__'@AmyTorresACIM'
Removing additional feature twitter__'@Amy__Cheong'
Removing additional feature twitter__'@AnadeaInc'
Removing additional feature twitter__'@AnaghaSathyapal'
Removing additional feature twitter__'@AnaisRaoux'
Removing additional feature twitter__'@AndreaCadelli'
Removing additional feature twitter__'@AndreaCarterINC'
Removing additional feature twitter__'@AndreasLeicher'
Removing additional feature twitter__'@AndreiFaji'
Removing additional feature twitter__'@Andrei_Calazans'
Removing additional feature twitter__'

Removing additional feature twitter__'@Bettymedia'
Removing additional feature twitter__'@Bhaskaratdaana'
Removing additional feature twitter__'@BiiG_Pongsatorn'
Removing additional feature twitter__'@BikoMandelaGray'
Removing additional feature twitter__'@BillLin0303'
Removing additional feature twitter__'@BilyanaVacheva'
Removing additional feature twitter__'@BinoyChitale'
Removing additional feature twitter__'@BirchStartups'
Removing additional feature twitter__'@Birdbassador'
Removing additional feature twitter__'@BischoffCh'
Removing additional feature twitter__'@BitClave'
Removing additional feature twitter__'@BitcoinPeso'
Removing additional feature twitter__'@Bizdynamicx'
Removing additional feature twitter__'@BlackHarpy'
Removing additional feature twitter__'@Blackabarbe'
Removing additional feature twitter__'@BlairImani'
Removing additional feature twitter__'@BlairMahoney'
Removing additional feature twitter__'@BlaireWard'
Removing additional feature twitter__'@BlasLoo'
Remov

Removing additional feature twitter__'@Chema_Juarez_'
Removing additional feature twitter__'@ChenhaoTan'
Removing additional feature twitter__'@ChiaraTermine'
Removing additional feature twitter__'@Chikwe_I'
Removing additional feature twitter__'@ChillyPriG'
Removing additional feature twitter__'@Chillyn555'
Removing additional feature twitter__'@ChipFranks'
Removing additional feature twitter__'@ChipsDunne'
Removing additional feature twitter__'@ChrisChiakulas'
Removing additional feature twitter__'@ChrisInSTL'
Removing additional feature twitter__'@ChrisLundkvist'
Removing additional feature twitter__'@ChrisMarchie'
Removing additional feature twitter__'@ChrisMurphyCT'
Removing additional feature twitter__'@ChrisPearso_'
Removing additional feature twitter__'@ChrisSpalton'
Removing additional feature twitter__'@ChrisWhiteHQ'
Removing additional feature twitter__'@ChristianDLSDK'
Removing additional feature twitter__'@ChromaticHQ'
Removing additional feature twitter__'@ChromiumDev'
Re

Removing additional feature twitter__'@DavidMcQueen'
Removing additional feature twitter__'@DavidNeilson3'
Removing additional feature twitter__'@DavidStrauss'
Removing additional feature twitter__'@Davidjahr1'
Removing additional feature twitter__'@DaviidStreit'
Removing additional feature twitter__'@DayoScript'
Removing additional feature twitter__'@DeanBokhari'
Removing additional feature twitter__'@Death_Ray'
Removing additional feature twitter__'@Deb4CongressNM'
Removing additional feature twitter__'@DebraLobel'
Removing additional feature twitter__'@DecentraIizing'
Removing additional feature twitter__'@DeepShitpod'
Removing additional feature twitter__'@DeepakChopra'
Removing additional feature twitter__'@DefendConst'
Removing additional feature twitter__'@DeleuGyN'
Removing additional feature twitter__'@Deliana_T'
Removing additional feature twitter__'@Delphy_org'
Removing additional feature twitter__'@Denis_A_Ivanov'
Removing additional feature twitter__'@DenissAlimovs'
Removi

...

Removing additional feature twitter__'@thechelseagrin'
Removing additional feature twitter__'@thecodegangblog'
Removing additional feature twitter__'@thecompanylab'
Removing additional feature twitter__'@thecoryreid'
Removing additional feature twitter__'@thecreatelabs'
Removing additional feature twitter__'@thedailybeast'
Removing additional feature twitter__'@thedancingflea'
Removing additional feature twitter__'@thedrbits'
Removing additional feature twitter__'@thedug'
Removing additional feature twitter__'@thefayth'
Removing additional feature twitter__'@thefrenkel'
Removing additional feature twitter__'@thegoodjungle'
Removing additional feature twitter__'@thejamesmcaulay'
Removing additional feature twitter__'@thejerrylu'
Removing additional feature twitter__'@thejohnprice'
Removing additional feature twitter__'@thejulieholmes'
Removing additional feature twitter__'@thekerp'
Removing additional feature twitter__'@thekingsindian'
Removing additional feature twitter__'@thekwakuel'


Removing additional feature twitter__'@tranvu'
Removing additional feature twitter__'@traveldeveloper'
Removing additional feature twitter__'@traversjul'
Removing additional feature twitter__'@travishines'
Removing additional feature twitter__'@travisjeffery'
Removing additional feature twitter__'@traviskellerman'
Removing additional feature twitter__'@trent_mclaren'
Removing additional feature twitter__'@trevin'
Removing additional feature twitter__'@treyathletes'
Removing additional feature twitter__'@tridipthrizu'
Removing additional feature twitter__'@trimmsler'
Removing additional feature twitter__'@trimmurrti'
Removing additional feature twitter__'@trinugraha'
Removing additional feature twitter__'@tripathy'
Removing additional feature twitter__'@tristanpope'
Removing additional feature twitter__'@tristanwatson'
Removing additional feature twitter__'@troynguyen'
Removing additional feature twitter__'@troysk704'
Removing additional feature twitter__'@true_tothemark'
Removing addit

Removing additional feature twitter__'@vischella'
Removing additional feature twitter__'@vivchauh'
Removing additional feature twitter__'@viveknb'
Removing additional feature twitter__'@vivekteega'
Removing additional feature twitter__'@viviant_g'
Removing additional feature twitter__'@vkozulya'
Removing additional feature twitter__'@vladimiroane'
Removing additional feature twitter__'@vlipaann'
Removing additional feature twitter__'@vljc17'
Removing additional feature twitter__'@voltera_io'
Removing additional feature twitter__'@volume88'
Removing additional feature twitter__'@voluntas'
Removing additional feature twitter__'@voordekunst'
Removing additional feature twitter__'@voteforryann'
Removing additional feature twitter__'@votem'
Removing additional feature twitter__'@vpaliy_'
Removing additional feature twitter__'@vrcraftauthor'
Removing additional feature twitter__'@vrepsys'
Removing additional feature twitter__'@vsmux'
Removing additional feature twitter__'@vtamethodman'
Remov

Removing additional feature twitter__'@yesmeck'
Removing additional feature twitter__'@yetimvakfi'
Removing additional feature twitter__'@yiminhye'
Removing additional feature twitter__'@yinonw'
Removing additional feature twitter__'@yndconsult'
Removing additional feature twitter__'@yogendra_h1'
Removing additional feature twitter__'@yomieluwande'
Removing additional feature twitter__'@yonigoldwasser'
Removing additional feature twitter__'@yoobi55'
Removing additional feature twitter__'@yordiverkroost'
Removing additional feature twitter__'@yorsofood'
Removing additional feature twitter__'@yoshi979'
Removing additional feature twitter__'@yosooff'
Removing additional feature twitter__'@yosoyene'
Removing additional feature twitter__'@yostane'
Removing additional feature twitter__'@yotako_io'
Removing additional feature twitter__'@yothinix'
Removing additional feature twitter__'@youearnedit'
Removing additional feature twitter__'@yourgirlhetty'
Removing additional feature twitter__'@you

,tag0,tag1,tag2,tag3,tag4,twitter__'@0guzKilic',twitter__'@1000YearYoung',twitter__'@101babich',twitter__'@1517fund',twitter__'@15Five',...,twitter__'@zellersamuel',twitter__'@zengularity',twitter__'@zenpeacekeeper',twitter__'@zephoria',twitter__'@zinegranada',twitter__'@zntfdr',twitter__'@zoyander',twitter__'@zs',twitter__'@zslayback',twitter__None
0,{'name':,"None,",'url':,"'https://medium.com/@HITRECORD.org',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,{'name':,"None,",'url':,"'https://medium.com/@mariabustillos',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,{'name':,"None,",'url':,"'https://medium.com/@HITRECORD.org',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,{'name':,"None,",'url':,"'https://medium.com/@LanceUlanoff',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,{'name':,"None,",'url':,"'https://blog.medium.com/@Medium',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
for col in cat_dummies:
    if col not in df_test_processed.columns:
        print("Adding missing feature {}".format(col))
        df_test_processed[col] = 0
df_test_processed.head()

Adding missing feature twitter__'@0723Sam'
Adding missing feature twitter__'@0canom'
Adding missing feature twitter__'@0ptionworkshop'
Adding missing feature twitter__'@0rf'
Adding missing feature twitter__'@0x13a'
Adding missing feature twitter__'@1000ant'
Adding missing feature twitter__'@1000currents'
Adding missing feature twitter__'@100Kin10'
Adding missing feature twitter__'@100dayswofear'
Adding missing feature twitter__'@108'
Adding missing feature twitter__'@120grammi'
Adding missing feature twitter__'@12KResearch'
Adding missing feature twitter__'@12hrdifference'
Adding missing feature twitter__'@148'
Adding missing feature twitter__'@16arPi'
Adding missing feature twitter__'@18havas'
Adding missing feature twitter__'@19223898'
Adding missing feature twitter__'@198Q'
Adding missing feature twitter__'@1FeaturedFollow'
Adding missing feature twitter__'@1_55_AM'
Adding missing feature twitter__'@1amnerd'
Adding missing feature twitter__'@1arsz'
Adding missing feature twitter__'@

Adding missing feature twitter__'@Agent47DarkSoul'
Adding missing feature twitter__'@AgileChrisYYC'
Adding missing feature twitter__'@AgileExistence'
Adding missing feature twitter__'@AgileLeanLife'
Adding missing feature twitter__'@Aglasser7205'
Adding missing feature twitter__'@AgoraForGood'
Adding missing feature twitter__'@AgoraOpinionCx'
Adding missing feature twitter__'@AguDeMarco'
Adding missing feature twitter__'@Agusfourquet'
Adding missing feature twitter__'@AhadIqbal'
Adding missing feature twitter__'@AhmedsKhutbahs'
Adding missing feature twitter__'@AidanMadiganC'
Adding missing feature twitter__'@Aiiane'
Adding missing feature twitter__'@Aimanisms'
Adding missing feature twitter__'@Aimee587'
Adding missing feature twitter__'@AineeWrites'
Adding missing feature twitter__'@AinsCas'
Adding missing feature twitter__'@AirFacts'
Adding missing feature twitter__'@AirbnbEng'
Adding missing feature twitter__'@AirliteGlobal'
Adding missing feature twitter__'@AirliteItalia'
Adding mi

Adding missing feature twitter__'@AniketGune'
Adding missing feature twitter__'@AninhaElisa'
Adding missing feature twitter__'@Anis_Malouche'
Adding missing feature twitter__'@Anita_Haak'
Adding missing feature twitter__'@AnnaFlagg'
Adding missing feature twitter__'@AnnaFokina42'
Adding missing feature twitter__'@AnnaOnTheWeb'
Adding missing feature twitter__'@AnnaStpnoff'
Adding missing feature twitter__'@AnneCecileGrbr'
Adding missing feature twitter__'@AnnePoirot'
Adding missing feature twitter__'@AnnickenDay'
Adding missing feature twitter__'@AnonyMousey'
Adding missing feature twitter__'@AnooshInTheSun'
Adding missing feature twitter__'@Anrick'
Adding missing feature twitter__'@Anthony'
Adding missing feature twitter__'@AnthonyAdams'
Adding missing feature twitter__'@AnthonyFrasier'
Adding missing feature twitter__'@AnthonyMarnell'
Adding missing feature twitter__'@Anthony_Bergen'
Adding missing feature twitter__'@Anthonymathiot'
Adding missing feature twitter__'@AntjePfahl'
Addin

Adding missing feature twitter__'@BeeckCenter'
Adding missing feature twitter__'@BeefyCrunch2017'
Adding missing feature twitter__'@BegumOztan'
Adding missing feature twitter__'@BehanceRussia'
Adding missing feature twitter__'@BeingCharley'
Adding missing feature twitter__'@BeingErnst'
Adding missing feature twitter__'@BeingLiberal'
Adding missing feature twitter__'@BelahdjiCm'
Adding missing feature twitter__'@Beldar'
Adding missing feature twitter__'@Beler'
Adding missing feature twitter__'@Bell97Bella'
Adding missing feature twitter__'@BellatorRex'
Adding missing feature twitter__'@BelleBCooper'
Adding missing feature twitter__'@Bellroy'
Adding missing feature twitter__'@Beloola'
Adding missing feature twitter__'@Beltrew'
Adding missing feature twitter__'@Bemorte'
Adding missing feature twitter__'@BenAustinBlog'
Adding missing feature twitter__'@BenBelchak'
Adding missing feature twitter__'@BenBostic'
Adding missing feature twitter__'@BenDressler'
Adding missing feature twitter__'@B

Adding missing feature twitter__'@BridgetMarie'
Adding missing feature twitter__'@BrightAgrotech'
Adding missing feature twitter__'@BrightFunnel'
Adding missing feature twitter__'@Brilliantcrank'
Adding missing feature twitter__'@BritainStays'
Adding missing feature twitter__'@BrittJMartin'
Adding missing feature twitter__'@BrizAndrew'
Adding missing feature twitter__'@BrockMeeks'
Adding missing feature twitter__'@Brock_toon'
Adding missing feature twitter__'@BrokeAssStuart'
Adding missing feature twitter__'@BrookeNedland'
Adding missing feature twitter__'@BrotherAmouzou'
Adding missing feature twitter__'@BrotherMob'
Adding missing feature twitter__'@Brother_sKeeper'
Adding missing feature twitter__'@BrownGirlMag'
Adding missing feature twitter__'@Brownstoner'
Adding missing feature twitter__'@BrsMncr'
Adding missing feature twitter__'@BruceMirken'
Adding missing feature twitter__'@BrunoBianchi23'
Adding missing feature twitter__'@BrunoFonzi'
Adding missing feature twitter__'@BryanJime

...

Adding missing feature twitter__'@utkukaynar'
Adding missing feature twitter__'@uvajfrank'
Adding missing feature twitter__'@uvarov916'
Adding missing feature twitter__'@ux_app'
Adding missing feature twitter__'@ux_art'
Adding missing feature twitter__'@uxconnections'
Adding missing feature twitter__'@uxdesignagency'
Adding missing feature twitter__'@uxdiogenes'
Adding missing feature twitter__'@uxmomo'
Adding missing feature twitter__'@uxpeek'
Adding missing feature twitter__'@uyangaa133'
Adding missing feature twitter__'@uzeconomist'
Adding missing feature twitter__'@uzrak'
Adding missing feature twitter__'@v_clayssen'
Adding missing feature twitter__'@v_conrado'
Adding missing feature twitter__'@v_solesmith'
Adding missing feature twitter__'@vaalpb'
Adding missing feature twitter__'@vabelmonte'
Adding missing feature twitter__'@vacord'
Adding missing feature twitter__'@vadimgrin'
Adding missing feature twitter__'@vahanalytics'
Adding missing feature twitter__'@vaibhav_khulbe'
Adding

Adding missing feature twitter__'@voxomalley'
Adding missing feature twitter__'@voxpluma'
Adding missing feature twitter__'@voxukraine'
Adding missing feature twitter__'@voyadi'
Adding missing feature twitter__'@voyageur_techno'
Adding missing feature twitter__'@vpax'
Adding missing feature twitter__'@vpetersson'
Adding missing feature twitter__'@vpisteve'
Adding missing feature twitter__'@vpreddy'
Adding missing feature twitter__'@vrana92'
Adding missing feature twitter__'@vrandezo'
Adding missing feature twitter__'@vrjump'
Adding missing feature twitter__'@vstudentmax'
Adding missing feature twitter__'@vt_olivier'
Adding missing feature twitter__'@vteurlay'
Adding missing feature twitter__'@vtruralbroadbnd'
Adding missing feature twitter__'@vvanedwards'
Adding missing feature twitter__'@vvanwilgenburg'
Adding missing feature twitter__'@vvchambers'
Adding missing feature twitter__'@vvorski'
Adding missing feature twitter__'@vwyoung'
Adding missing feature twitter__'@vytautasalech'
Add

Adding missing feature twitter__'@wlonk'
Adding missing feature twitter__'@wmertens'
Adding missing feature twitter__'@wmilam'
Adding missing feature twitter__'@wnascimentow'
Adding missing feature twitter__'@wnialloconnor'
Adding missing feature twitter__'@wntart'
Adding missing feature twitter__'@wojtekidd'
Adding missing feature twitter__'@woke_nation'
Adding missing feature twitter__'@woldchar'
Adding missing feature twitter__'@wolframnagel'
Adding missing feature twitter__'@wolframthurm'
Adding missing feature twitter__'@wolfton'
Adding missing feature twitter__'@wolox'
Adding missing feature twitter__'@wond3rjay'
Adding missing feature twitter__'@wonderpaolastra'
Adding missing feature twitter__'@wonderstuck'
Adding missing feature twitter__'@wongweezy'
Adding missing feature twitter__'@wooky2000'
Adding missing feature twitter__'@wootric'
Adding missing feature twitter__'@wordcandy_co'
Adding missing feature twitter__'@wordsbydan'
Adding missing feature twitter__'@worduct'
Addin

Adding missing feature twitter__'@yuvizalkow'
Adding missing feature twitter__'@yuzun_m'
Adding missing feature twitter__'@yvesmariecann'
Adding missing feature twitter__'@yvonniks'
Adding missing feature twitter__'@yvoschaap'
Adding missing feature twitter__'@ywxwy'
Adding missing feature twitter__'@zach_cmiel'
Adding missing feature twitter__'@zach_two_times'
Adding missing feature twitter__'@zachbeachlove'
Adding missing feature twitter__'@zachinglis'
Adding missing feature twitter__'@zachklein'
Adding missing feature twitter__'@zachluz'
Adding missing feature twitter__'@zachmathew'
Adding missing feature twitter__'@zachstechturf'
Adding missing feature twitter__'@zachzamboni'
Adding missing feature twitter__'@zack'
Adding missing feature twitter__'@zackbaddorf'
Adding missing feature twitter__'@zackmiller84'
Adding missing feature twitter__'@zackpennington'
Adding missing feature twitter__'@zackrisson'
Adding missing feature twitter__'@zackzadek'
Adding missing feature twitter__'@z

,tag0,tag1,tag2,tag3,tag4,twitter__'@0guzKilic',twitter__'@1000YearYoung',twitter__'@101babich',twitter__'@1517fund',twitter__'@15Five',...,twitter__'@zubaortiz',twitter__'@zumpang',twitter__'@zvellas',twitter__'@zverk0',twitter__'@zysman',twitter__'@zzam_',twitter__'@zzbennett',twitter__'@zzdoublezz',twitter__'@zzilch',twitter__'@zzste'
0,{'name':,"None,",'url':,"'https://medium.com/@HITRECORD.org',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,{'name':,"None,",'url':,"'https://medium.com/@mariabustillos',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,{'name':,"None,",'url':,"'https://medium.com/@HITRECORD.org',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,{'name':,"None,",'url':,"'https://medium.com/@LanceUlanoff',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,{'name':,"None,",'url':,"'https://blog.medium.com/@Medium',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [105]:
df_test_processed = df_test_processed[processed_columns]
df_test_processed.head()

,tag0,tag1,tag2,tag3,tag4,twitter__'@0723Sam',twitter__'@0canom',twitter__'@0guzKilic',twitter__'@0ptionworkshop',twitter__'@0rf',...,twitter__'@zumpang',twitter__'@zvellas',twitter__'@zverk0',twitter__'@zysman',twitter__'@zzam_',twitter__'@zzbennett',twitter__'@zzdoublezz',twitter__'@zzilch',twitter__'@zzste',twitter__None
0,{'name':,"None,",'url':,"'https://medium.com/@HITRECORD.org',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,{'name':,"None,",'url':,"'https://medium.com/@mariabustillos',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,{'name':,"None,",'url':,"'https://medium.com/@HITRECORD.org',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,{'name':,"None,",'url':,"'https://medium.com/@LanceUlanoff',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,{'name':,"None,",'url':,"'https://blog.medium.com/@Medium',",'twitter':,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [106]:
df_author_processed = df_author_processed.drop(columns=['tag0', 'tag1', 'tag2', 'tag3', 'tag4'])
X_train_author_sparse = df_author_processed.values
df_test_processed = df_test_processed.drop(columns=['tag0', 'tag1', 'tag2', 'tag3', 'tag4'])
X_test_author_sparse = df_test_processed.values
print(X_train_author_sparse.shape)
print(X_test_author_sparse.shape)

(62313, 23588)
(34645, 23588)


In [107]:
np.save(os.path.join(PATH_TO_DATA, 'X_train_author_sparse.npy'), X_train_author_sparse)
np.savetxt(os.path.join(PATH_TO_DATA, 'X_test_author_sparse.npy'), X_test_author_sparse)

OSError: [Errno 28] No space left on device

**Join all sparse matrices.**

In [71]:
X_train_sparse = hstack([X_train_content_sparse, X_train_title_sparse,
                         X_train_author_sparse, 
                         X_train_time_features_sparse]).tocsr()

ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 62540, expected 62313.

In [ ]:
X_test_sparse = hstack([X_test_content_sparse, X_test_title_sparse,
                        X_test_author_sparse, 
                        X_test_time_features_sparse]).tocsr()

**Read train target and split data for validation.**

In [ ]:
train_target = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_log1p_recommends.csv'), 
                           index_col='id')
y_train = train_target['log_recommends'].values

In [ ]:
train_part_size = int(0.7 * train_target.shape[0])
X_train_part_sparse = X_train_sparse[:train_part_size, :]
y_train_part = y_train[:train_part_size]
X_valid_sparse =  X_train_sparse[train_part_size:, :]
y_valid = y_train[train_part_size:]

**Train a simple Ridge model and check MAE on the validation set.**

In [ ]:
# You code here

**Train the same Ridge with all available data, make predictions for the test set and form a submission file.**

In [ ]:
# You code here

In [ ]:
def write_submission_file(prediction, filename,
                          path_to_sample=os.path.join(PATH_TO_DATA, 
                                                      'sample_submission.csv')):
    submission = pd.read_csv(path_to_sample, index_col='id')
    
    submission['log_recommends'] = prediction
    submission.to_csv(filename)

In [ ]:
write_submission_file(ridge_test_pred, os.path.join(PATH_TO_DATA,
                                                    'assignment6_medium_submission.csv'))

**Now's the time for dirty Kaggle hacks. Form a submission file with all zeroes. Make a submission. What do you get if you think about it? How is it going to help you with modifying your predictions?**

In [ ]:
write_submission_file(np.zeros_like(ridge_test_pred), 
                      os.path.join(PATH_TO_DATA,
                                   'medium_all_zeros_submission.csv'))

**Modify predictions in an appropriate way (based on your all-zero submission) and make a new submission.**

In [ ]:
ridge_test_pred_modif = ridge_test_pred # You code here

In [ ]:
write_submission_file(ridge_test_pred_modif, 
                      os.path.join(PATH_TO_DATA,
                                   'assignment6_medium_submission_with_hack.csv'))

That's it for the assignment. Much more credits will be given to the winners in this competition, check [course roadmap](https://mlcourse.ai/roadmap). Do not spoil the assignment and the competition - don't share high-performing kernels (with MAE < 1.5).

Some ideas for improvement:

- Engineer good features, this is the key to success. Some simple features will be based on publication time, authors, content length and so on
- You may not ignore HTML and extract some features from there
- You'd better experiment with your validation scheme. You should see a correlation between your local improvements and LB score
- Try TF-IDF, ngrams, Word2Vec and GloVe embeddings
- Try various NLP techniques like stemming and lemmatization
- Tune hyperparameters. In our example, we've left only 50k features and used C=1 as a regularization parameter, this can be changed
- SGD and Vowpal Wabbit will learn much faster
- Play around with blending and/or stacking. An intro is given in [this Kernel](https://www.kaggle.com/kashnitsky/ridge-and-lightgbm-simple-blending) by @yorko 
- In our course, we don't cover neural nets. But it's not obliged to use GRUs/LSTMs/whatever in this competition.

Good luck!

<img src='../../img/kaggle_shakeup.png' width=50%>